<a href="https://colab.research.google.com/github/MooNight009/ColabProjects/blob/master/DONE_practice_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

print("Tensorflow version:", tf.__version__)
print("Eager:", "true" if tf.executing_eagerly else "false")
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Tensorflow version: 2.1.0-rc1
Eager: true
GPU is available


In [0]:
#Read the data
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [0]:
#Shuffle the data
california_housing_dataframe= california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [0]:
# X= california_housing_dataframe["total_rooms", "population", "total_bedrooms", "median_income"].values
X= california_housing_dataframe[['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']].values
Y= california_housing_dataframe["median_house_value"].values/ 10000.0

# X= np.reshape(X, (-1, 1))
Y= np.reshape(Y, (-1, 1))

train_features , test_features ,train_labels, test_labels = train_test_split( X , Y , test_size=0.2)

X= tf.constant(train_features, dtype=X.dtype)
# X= normalize(X)
Y= tf.constant(train_labels, dtype=Y.dtype)
# Y= normalize(Y)

test_X= tf.constant(test_features, dtype=X.dtype)
# test_X= normalize(test_X)
test_Y= tf.constant(test_labels, dtype=Y.dtype)
# test_Y= normalize(test_Y)

In [0]:
#Normalize the data
def normalize(data):
  mean, var = tf.nn.moments(data, [0], keepdims= True)
  new_tensor = tf.divide(tf.subtract(data, mean), tf.sqrt(var))
  return new_tensor

In [0]:
def mean_squared_error( Y , y_pred ):
    return tf.reduce_mean( tf.square( y_pred - Y ) )

def mean_squared_error_deriv( Y , y_pred ):
    return tf.reshape( tf.reduce_mean( 2 * ( y_pred - Y ) ) , [ 1 , 1 ] )
    
def h ( X , weights , bias ):
    return tf.tensordot( X , weights , axes=1 ) + bias

In [0]:
num_epochs= 20
num_samples= X.shape[0]
batch_size= 10
learning_rate= 0.001

dataset= tf.data.Dataset.from_tensor_slices((X, Y))
dataset= dataset.shuffle(500).repeat(num_epochs).batch(batch_size)
iterator= dataset.__iter__()

In [0]:
num_features = X.shape[1] if len(X.shape)> 1 else 1
weights = tf.random.normal( ( num_features , 1), dtype=X.dtype)

bias= 0

epochs_plot= list()
loss_plot= list()

for i in range(num_epochs):
  epoch_loss= list()
  
  for b in range(int(num_samples/ batch_size)):
    x_batch, y_batch= iterator.get_next()

    output = h( x_batch , weights , bias ) 
    loss = epoch_loss.append( mean_squared_error( y_batch , output ).numpy() )

    dJ_dH = mean_squared_error_deriv( y_batch , output)
    dH_dW = x_batch
    dJ_dW = tf.reduce_mean( dJ_dH * dH_dW )
    dJ_dB = tf.reduce_mean( dJ_dH )

    

    weights -= ( learning_rate * dJ_dW )
    bias -= ( learning_rate * dJ_dB ) 

    
    # if b< 2:
    #   print("Loss", mean_squared_error( y_batch , output ).numpy())
    #   print("dJ_dH", dJ_dH)
    #   print("Weight", weights)
    #   print("Bias", bias)
    #   print("Input", x_batch[:2])
    #   print("Output", output[:2])
    #   print("Expected", y_batch[:2])


    #   print("----------------")
  # print(epoch_loss)
  loss= np.array(epoch_loss).mean()
  epochs_plot.append(i+ 1)
  loss_plot.append(loss)

  print("Loss is {}".format(loss))

Loss is nan


KeyboardInterrupt: ignored

In [0]:
plt.plot(epochs_plot, loss_plot)
plt.show()

In [0]:
output = h( test_X , weights , bias ) 
labels = test_Y

accuracy_op = tf.metrics.MeanAbsoluteError() 
accuracy_op.update_state( labels , output )
print( 'Mean Absolute Error = {}'.format( accuracy_op.result().numpy() ) )

accuracy_op = tf.metrics.RootMeanSquaredError() 
accuracy_op.update_state( labels , output )
print( 'Root Mean Square Error = {}'.format( accuracy_op.result().numpy() ) )

--------------------------------------------------------------------------------
---
---
---
---
This uses estimator


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.compat.v1.data as tfv1
from sklearn import metrics
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

print("Tensorflow version:", tf.__version__)
print("Eager:", "true" if tf.executing_eagerly else "false")
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Tensorflow version: 2.1.0-rc1
Eager: true
GPU is available


In [0]:
#Read the data
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [0]:
#Shuffle the data
california_housing_dataframe= california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207.300912
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115.983764
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14.999000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119.400000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180.400000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500.001000


In [0]:
# features= california_housing_dataframe[['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']]
input= ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
my_features= california_housing_dataframe[input]
# feature_columns = [tf.feature_column.numeric_column('housing_median_age'), tf.feature_column.numeric_column('total_rooms')]
feature_columns = [tf.feature_column.numeric_column(x) for x in input]

targets= california_housing_dataframe["median_house_value"]



In [0]:
#Optimizer
my_optimizer= tf.optimizers.Adam(learning_rate=0.001)

# linear_regressor= tf.estimator.LinearRegressor(
linear_regressor= tf.estimator.DNNLinearCombinedRegressor(
    # feature_columns= feature_columns,
    linear_feature_columns= feature_columns,
    dnn_optimizer= my_optimizer
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpomz_5dbn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
def my_input_fn(features, targets, batch_size= 1, shuffle= True, num_epochs= None):
  features= {key:np.array(value) for key, value in dict(features).items()}

  dataset= tfv1.Dataset.from_tensor_slices((features, targets))
  dataset= dataset.batch(batch_size).repeat(num_epochs)

  if shuffle:
    dataset= dataset.shuffle(buffer_size= 10000)

  # iter= eval_input_fn()
  # return next(iter)
  # return next(dataset.__iter__())
  # iterator= dataset.__iter__()
  
  features, labels = dataset.make_one_shot_iterator().get_next()
  return features, labels

In [0]:
linear_regressor.train(
    input_fn = lambda:my_input_fn(my_features, targets, num_epochs= 10),
    steps= 500
)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpomz_5dbn/model.ckpt.
INFO:tensorflow:loss = 13665.61, step = 0
INFO:tensorflow:global_step/sec: 304

In [0]:
predictions = linear_regressor.predict(input_fn=lambda: my_input_fn(my_features, targets, batch_size= 1, num_epochs=1, shuffle=False))

predictions = np.array([item['predictions'][0] for item in predictions])

mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpomz_5dbn/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Mean Squared Error (on training data): 27278.923
Root Mean Squared Error (on training data): 165.163


--------------------------------------------------------------------------------
---
---
---
---
This uses simple neural net


In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

print("Tensorflow version:", tf.__version__)
print("Eager:", "true" if tf.executing_eagerly else "false")
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Tensorflow version: 2.1.0-rc1
Eager: true
GPU is available


In [0]:
#Read the data
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [0]:
#Shuffle the data
california_housing_dataframe= california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [0]:
# X= california_housing_dataframe["total_rooms", "population", "total_bedrooms", "median_income"].values
X= california_housing_dataframe[['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']].values
Y= california_housing_dataframe["median_house_value"].values/ 10000.0

# X= np.reshape(X, (-1, 1))
Y= np.reshape(Y, (-1, 1))

train_features , test_features ,train_labels, test_labels = train_test_split( X , Y , test_size=0.2)


In [0]:
model= tf.keras.Sequential(){}
model.add(tf.keras.layers.Dense(6, activation= 'relu'))
model.add(tf.keras.layers.Dense(4, activation= 'relu'))
model.add(tf.keras.layers.Dense(1, activation= 'relu'))

model.summary()

ValueError: ignored

In [0]:
model.compile(
    optimizer= 'adam',
    loss='mean_squared_error'
)

trained= model.fit(x= train_features, y= train_labels, epochs= 20, verbose= 1)

Train on 13600 samples
Epoch 1/20
13600/13600 [==============================] - 1s 95us/sample - loss: 9573.7766
Epoch 2/20
13600/13600 [==============================] - 1s 75us/sample - loss: 575.3915
Epoch 3/20
13600/13600 [==============================] - 1s 78us/sample - loss: 569.0013
Epoch 4/20
13600/13600 [==============================] - 1s 78us/sample - loss: 566.1147
Epoch 5/20
13600/13600 [==============================] - 1s 76us/sample - loss: 565.3134
Epoch 6/20
13600/13600 [==============================] - 1s 78us/sample - loss: 565.2376
Epoch 7/20
13600/13600 [==============================] - 1s 77us/sample - loss: 565.2390
Epoch 8/20
13600/13600 [==============================] - 1s 80us/sample - loss: 565.2416
Epoch 9/20
13600/13600 [==============================] - 1s 77us/sample - loss: 565.2633
Epoch 10/20
13600/13600 [==============================] - 1s 78us/sample - loss: 565.2340
Epoch 11/20
13600/13600 [==============================] - 1s 77us/sample -

In [0]:
evaluation= model.evaluate(x= test_features, y= test_labels, batch_size= 512, verbose= 1)
print(evaluation)
# for name, value in zip(model.metrics_names, evaluation):
#   print("%s: %0.3f" %(name, value))

3400/3400 [==============================] - 0s 21us/sample - loss: 559.8810
559.8810146197151


--------------------------------------------------------------------------------
---
---
---
---
This uses sklearn


In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

print("Tensorflow version:", tf.__version__)
print("Eager:", "true" if tf.executing_eagerly else "false")
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

Tensorflow version: 2.1.0-rc1
Eager: true
GPU is available


In [0]:
reg= LinearRegression().fit(train_features, train_labels)

output= reg.predict(test_features)

accuracy_op = tf.metrics.MeanAbsoluteError() 
accuracy_op.update_state( test_labels , output )
print( 'Mean Absolute Error = {}'.format( accuracy_op.result().numpy() ) )

accuracy_op = tf.metrics.RootMeanSquaredError() 
accuracy_op.update_state( test_labels , output )
print( 'Root Mean Square Error = {}'.format( accuracy_op.result().numpy() ) )

Mean Absolute Error = 5.515396595001221
Root Mean Square Error = 7.550936222076416
